In [ ]:
! pip install langchain wikipedia langchain-community tavily-python langchainhub langchain-openai openai chromadb

In [ ]:
#!pip install langchain langchain_openai langchainhub chromadb

from google.colab import userdata
secret = userdata.get('OPENAI_TOKEN')

# Load docs
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://georgewbush-whitehouse.archives.gov/news/releases/2003/01/20030128-19.es.html")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(openai_api_key=secret))

# RAG prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# LLM
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=secret)

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)
question = "¿Qué conclusiones se obtienen sobre Iraq?"
result = qa_chain({"query": question})
result["result"]
